## Observations and Insights 

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "data/Mouse_metadata.csv"
study_results_path = "data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)


In [ ]:
# Combine the data into a single dataset
mouse_df = pd.merge(study_results, mouse_metadata, on="Mouse ID")

In [ ]:
# Display the data table for preview
mouse_df.head()

In [ ]:
mouse_df.info()

In [ ]:
# Checking the number of mice.
len(pd.unique(mouse_df['Mouse ID']))

In [ ]:
# Getting the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
dupes = mouse_df.groupby(["Mouse ID", "Timepoint"]).size().reset_index()
dupes.columns = ["Mouse ID", "Timepoint", "Count"]
dupes.sort_values(by="Count", ascending=False).head()

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
mouse_df.loc[mouse_df["Mouse ID"] == "g989"]

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
mouse_drop_df = mouse_df.loc[mouse_df["Mouse ID"] != "g989"]
mouse_drop_df

In [ ]:
# Checking the number of mice in the clean DataFrame.
len(pd.unique(mouse_drop_df['Mouse ID']))

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary dataframe.

reg_mean = mouse_drop_df.groupby("Drug Regimen").mean()
reg_median = mouse_drop_df.groupby("Drug Regimen").median()
reg_var = mouse_drop_df.groupby("Drug Regimen").var()
reg_std = mouse_drop_df.groupby("Drug Regimen").std()
reg_sem = mouse_drop_df.groupby("Drug Regimen").sem()

In [ ]:
# Selecting Tumor Volume column
reg_mean = mouse_drop_df.groupby("Drug Regimen")["Tumor Volume (mm3)"].mean()
reg_median = mouse_drop_df.groupby("Drug Regimen")["Tumor Volume (mm3)"].median()
reg_var = mouse_drop_df.groupby("Drug Regimen")["Tumor Volume (mm3)"].var()
reg_std = mouse_drop_df.groupby("Drug Regimen")["Tumor Volume (mm3)"].std()
reg_sem = mouse_drop_df.groupby("Drug Regimen")["Tumor Volume (mm3)"].sem()

In [ ]:
summary = pd.DataFrame(reg_mean)
summary.rename(columns = {'Tumor Volume (mm3)':'Tumor Mean'}, inplace=True)
summary["Tumor Median"] = pd.DataFrame(reg_median)
summary["Tumor Var"] = pd.DataFrame(reg_var)
summary["Tumor Std"] = pd.DataFrame(reg_std)
summary["Tumor SEM"] = pd.DataFrame(reg_sem)
summary

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Using the aggregation method, produce the same summary statistics in a single line

mouse_drop_df.groupby("Drug Regimen")["Tumor Volume (mm3)"].agg(["mean", "median", "var", "std", "sem"])

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pandas.

mouse_count = mouse_drop_df.groupby("Drug Regimen")["Mouse ID"].nunique()
mouse_count = pd.DataFrame(mouse_count)
mouse_count.rename(columns={"Mouse ID": "Mouse Count"}, inplace=True)
mouse_count

In [ ]:
mouse_count.plot(kind="bar", color=['purple']).legend(loc='lower right')

plt.title("Number of Unique Mice Tested per Regimen")
plt.xlabel("Regimen")
plt.ylabel("Mouse Count")

In [ ]:
mouse_count = mouse_count.reset_index()

In [ ]:
# Generate a bar plot showing the total number of unique mice tested on each drug regimen using pyplot.

plt.figure(figsize=(10,6))
plt.bar(mouse_count["Drug Regimen"], mouse_count["Mouse Count"], color="teal")

plt.title("Number of Unique Mice Tested per Regimen", fontsize=20, fontweight="bold")
plt.xlabel("Drug Regimen", fontsize=14)
plt.ylabel("Number of Mice", fontsize=14)

plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pandas

mouse_gen = mouse_drop_df.loc[:, ["Mouse ID", "Sex"]].drop_duplicates()
mouse_gen_count = mouse_gen.groupby("Sex").count()
mouse_gen_count.rename(columns={"Mouse ID": "Mouse Count"}, inplace=True)
mouse_gen_count

In [ ]:
mouse_gen_count.plot(kind="pie", subplots=True, autopct='%1.1f%%', figsize=(8, 8))

In [ ]:
mouse_gen_count.index

In [ ]:
mouse_gen_count["Mouse Count"].values

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot

plt.figure(figsize=(10,6))
plt.pie(mouse_gen_count["Mouse Count"].values, labels=mouse_gen_count.index, colors=["pink", "lightskyblue"], autopct="%1.1f%%")

# Create axes which are equal so we have a perfect circle
plt.axis("equal")

plt.title("Mouse Gender Breakdown", fontsize=20, fontweight="bold")
plt.show()

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original dataframe to get the tumor volume at the last timepoint


In [ ]:
final_volume = mouse_drop_df.groupby(["Mouse ID", "Drug Regimen"])
final_volume = final_volume["Timepoint"].max()
final_volume = final_volume.reset_index()
final_volume2 = pd.merge(final_volume, mouse_drop_df, on=["Mouse ID", "Drug Regimen", "Timepoint"], how="left")
final_volume2.head()

In [ ]:
drug_list = ["Capomulin", "Ramicane", "Infubinol", "Ceftamin"]
final_volume3 = final_volume2[final_volume2["Drug Regimen"].isin(drug_list)].reset_index(drop=True)
final_volume3.head()

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)

treatment = final_volume3["Drug Regimen"].tolist()

# Create empty list to fill with tumor vol data (for plotting)

tumor_cap = []
tumor_ram = []
tumor_inf = []
tumor_cef = []

# Calculate the IQR and quantitatively determine if there are any potential outliers. 

# quartiles = final_volume3["Tumor Volume (mm3)"].quantile([.25,.5,.75])
# lowerq = quartiles[0.25]
# upperq = quartiles[0.75]
# iqr = upperq-lowerq

    # Locate the rows which contain mice on each drug and get the tumor volumes

for row in range(len(final_volume3)):
    if final_volume3.loc[row, "Drug Regimen"] == "Capomulin":
        tumor_cap.append(final_volume3.loc[row, "Tumor Volume (mm3)"])
    elif final_volume3.loc[row, "Drug Regimen"] == "Ramicane":
        tumor_ram.append(final_volume3.loc[row, "Tumor Volume (mm3)"])
    elif final_volume3.loc[row, "Drug Regimen"] == "Infubinol":
        tumor_inf.append(final_volume3.loc[row, "Tumor Volume (mm3)"])
    else:
        tumor_cef.append(final_volume3.loc[row, "Tumor Volume (mm3)"])
       
    # add subset 

tumor_iqr_df = pd.DataFrame([tumor_cap, tumor_ram, tumor_inf, tumor_cef]).T
tumor_iqr_df.columns = ["Cap_Volume", "Ram_Volume", "Inf_Volume", "Cef_Volume"]

# tumor_cap_df = pd.DataFrame(tumor_cap) 
# tumor_cap_df.columns = ["Volume"]
    
    # Determine outliers using upper and lower bounds

# quartiles = tumor_cap_df["Volume"].quantile([.25,.5,.75])
lowerq = tumor_iqr_df.quantile(0.25)
upperq = tumor_iqr_df.quantile(0.75)
iqr = upperq-lowerq
    
lower_bound = lowerq - (1.5*iqr)
upper_bound = upperq + (1.5*iqr)

outlier = tumor_iqr_df[((tumor_iqr_df < lower_bound) | (tumor_iqr_df > upper_bound))]
outlier
# outlier = tumor_iqr_df[(tumor_cap_df < lower_bound) | (tumor_cap_df > upper_bound)]
# outlier

In [ ]:
# Verifying outliers
print(f"Values below {lower_bound} could be outliers.")
print(f"Values above {upper_bound} could be outliers.")

In [ ]:
# Generate a box plot of the final tumor volume of each mouse across four regimens of interest
fig1, ax1 = plt.subplots()
ax1.set_title("Total Volume per Drug Regimen")
ax1.set_ylabel("Volume")
ax1.boxplot(tumor_iqr_df, labels=["Capomulin", "Ramicane", "Infubinol", "Ceftamin"],
            flierprops={'marker': 'o', 'markersize': 10, 'markerfacecolor': 'red'})
plt.show()

## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a mouse treated with Capomulin
mouse_line = mouse_drop_df.loc[mouse_df["Mouse ID"] == "m601"]
mouse_line

In [ ]:
plt.plot(mouse_line["Timepoint"], mouse_line["Tumor Volume (mm3)"])
plt.title("Tumor Volume Treated by Capomulin", fontsize=20, fontweight="bold")
plt.xlabel("Timepoint", fontsize=14)
plt.ylabel("Tumor Volume (mm3)", fontsize=14)
plt.show()

In [ ]:
# Generate a scatter plot of average tumor volume vs. mouse weight for the Capomulin regimen
mouse_scatter = mouse_drop_df.loc[mouse_df["Drug Regimen"] == "Capomulin"]
mouse_avg_vol = mouse_scatter.groupby("Mouse ID")["Tumor Volume (mm3)"].mean().tolist()

In [ ]:
mouse_weight = mouse_scatter.groupby("Mouse ID")
mouse_weight = mouse_weight["Weight (g)"].first().tolist()

In [ ]:
plt.title("Avgerage Tumor Volume v Weight for Capomulin Regimen")
plt.xlabel("Weight (g)")
plt.ylabel("Average Tumor Volume (mm3)")
plt.scatter(mouse_weight, mouse_avg_vol, marker="o", facecolors="red", edgecolors="black")

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and linear regression model 
# for mouse weight and average tumor volume for the Capomulin regimen
correlation = st.pearsonr(mouse_weight, mouse_avg_vol)
print(f"The correlation between both factors is {round(correlation[0],2)}")

In [ ]:
reg_data = pd.DataFrame()
mouse_weight = mouse_scatter.groupby("Mouse ID")
reg_data = pd.DataFrame(mouse_weight["Weight (g)"].first())

In [ ]:
mouse_scatter = mouse_drop_df.loc[mouse_df["Drug Regimen"] == "Capomulin"]
reg_data["Average Tumor Volume (mm3)"] = mouse_scatter.groupby("Mouse ID")["Tumor Volume (mm3)"].mean()
reg_data["Weight (g)"] = reg_data["Weight (g)"].astype(float)

In [ ]:
x_values = reg_data["Weight (g)"]
y_values = reg_data["Average Tumor Volume (mm3)"]

slope, intercept, rvalue, pvalue, stderr = st.linregress(x_values, y_values)

regress_values = x_values * slope + intercept
line_eq = "y = " + str(round(slope,2)) + "x + " + str(round(intercept,2))
plt.scatter(x_values,y_values)
plt.plot(x_values,regress_values,"r-")
plt.annotate(line_eq,(20,38),fontsize=15,color="red")
plt.xlabel("Weight (g)")
plt.ylabel("Average Tumor Volume (mm3)")
plt.show()

In [ ]:
# observations and insights
'''
The boxplot shows the consistency of the drug regimens having only one outlier. Also, Infubinol and Ceftamin were used to treat
the mice with higher tumor volume while Capomulin and Ramicane took care of those with lower tumor volume.

In focusing on the Capomulin regimen, the treatment reduces the tumor volume over time based on the line plot.

At first glance, the scatter plot shows a positive correlation between weight and average tumor volume. That observation is
verified having a correlation coefficient of .84 and a very positive slope in the linear regression model.
'''